In [2]:
import math
import random
import os
import numpy as np

In [3]:
def calculate_LJ(r_ij):
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy

In [4]:
def calculate_distance(coord1, coord2, box_length=None):

    distance = 0
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i]) 
        
        if box_length:
            dim_dist = dim_dist - box_length * round(dim_dist / box_length)
        
        dim_dist = dim_dist**2
        distance += dim_dist
    
    distance = math.sqrt(distance)
    return distance

In [5]:
def calculate_tail_correction(num_particles, box_length, cutoff):
    const1 = (8 * math.pi * num_particles ** 2) / (3 * box_length ** 3)
    const2 = (1/3) * (1 / cutoff)**9 - (1 / cutoff) **3
    
    return const1 * const2

In [6]:
def calculate_total_energy(coordinates, box_length, cutoff): 
    total_energy = 0
    
    num_atoms = len(coordinates)
    
    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            
            #print(f'Comparings atom number {i} with atom number {j}')
            
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length=box_length)
            
            if dist_ij < cutoff:
                interaction_energy = calculate_LJ(dist_ij)
                total_energy += interaction_energy
            
    return total_energy

In [7]:
def read_xyz(filepath):
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length

In [8]:
def accept_or_reject(delta_e, beta):
    
    
    if delta_e <= 0:
        accept = True
    else:
        random_number = random.random()
        p_acc = math.exp(-beta*delta_e)
        
        if random_number < p_acc:
            accept = True
        else:
            accept = False
    
    return accept

In [9]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    
    
    e_total = 0
    
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
        if i_particle != j_particle:
            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position, box_length)

            if rij < cutoff:
                e_pair = calculate_LJ(rij)
                e_total += e_pair
    
    return e_total

In [10]:
#calculate_p
def calculate_f(r_ij):
    
    r6_term = np.power(1/r_ij, 6)
    r12_term = np.power(r6_term, 2)
    
    f = 48 * (r12_term - 0.5*r6_term)/(r_ij**2) * r_ij
    
    return f

In [11]:
def calculate_tail_p(num_particles, box_length, r_ij):
    
    
    const = (16 * np.pi * num_particles ** 2) / (3 * box_length ** 6)
    r3_term = 1/(r_ij**3)
    r9_term = 2/3*(r3_term**3)
    p_tail = const *(r9_term - r3_term)
    
    return p_tail

In [12]:
def calculate_total_p(T, coordinates, box_length, cutoff):
    
    num_atoms = len(coordinates)
    sum_fr = 0
    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length=box_length)
            if dist_ij < cutoff:
                interaction_f = calculate_f(dist_ij)
                fr = interaction_f * dist_ij
                sum_fr = sum_fr+fr
    total_p = (3*num_atoms*T+sum_fr)/(3*box_length**3)    
    return total_p

In [13]:
def mc_energy_pressure(reduced_temperature,num_steps,max_displacement,cutoff,freq,file_path):

    steps = []
    energies = []
    all_coordinates = []

    # Calculated quantities
    beta = 1/reduced_temperature

    # Read initial coordinates
    coordinates, box_length = read_xyz(file_path)
    num_particles = len(coordinates)

    # Calculated based on simulation inputs
    total_energy = calculate_total_energy(coordinates, box_length, cutoff)
    total_energy += calculate_tail_correction(num_particles, box_length, cutoff)
    print('ori_energy')
    print(total_energy/num_particles)
    total_pressure_notail = calculate_total_p(reduced_temperature, coordinates, box_length, cutoff)
    tail_pressure = calculate_tail_p(num_particles, box_length, cutoff)
    total_pressure = total_pressure_notail+tail_pressure
    print('ori_pressure')
    print(total_pressure)
    print('-------------------')
    for step in range(num_steps):

        # 1. Randomly pick one of num_particles particles
        random_particle = random.randrange(num_particles)

        # 2. Calculate the interaction energy of the selected particle with the system. Store this value.
        current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)

        # 3. Generate a random x, y, z displacement range (-max_displacement, max_displacement) - uniform distribution
        x_rand = random.uniform(-max_displacement, max_displacement)
        y_rand = random.uniform(-max_displacement, max_displacement)
        z_rand = random.uniform(-max_displacement, max_displacement)

        # 4. Modify the coordinate of selected particle by generated displacements.
        coordinates[random_particle][0] += x_rand
        coordinates[random_particle][1] += y_rand
        coordinates[random_particle][2] += z_rand

        # 5. Calculate the new interaction energy of moved particle, store this value.
        proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)

        # 6. Calculate energy change and decide if we accept the move.
        delta_energy = proposed_energy - current_energy

        accept = accept_or_reject(delta_energy, beta)

        # 7. If accept, keep movement. If not revert to old position.
        if accept:
            total_energy += delta_energy
            total_pressure_notail = calculate_total_p(reduced_temperature, coordinates, box_length, cutoff)
            tail_pressure = calculate_tail_p(num_particles, box_length, cutoff)
            total_pressure = total_pressure_notail+tail_pressure
        else:
            # Move is not accepted, roll back coordinates
            coordinates[random_particle][0] -= x_rand
            coordinates[random_particle][1] -= y_rand
            coordinates[random_particle][2] -= z_rand

        # 8. Print the energy and store the coordinates at certain intervals
        if step % freq == 0:
            print(step, total_energy/num_particles,total_pressure)
            print('-------------------')
            steps.append(step)
            energies.append(total_energy/num_particles)
            all_coordinates.append(coordinates)

In [14]:
reduced_temperature = 1.5
num_steps = 50000
max_displacement = 0.1 
cutoff = 3
freq = 1000
file_path = r'C:\Users\91095\Desktop\xyz_file\lj_sample_config_periodic1(1).txt'
mc_energy_pressure(reduced_temperature,num_steps,max_displacement,cutoff,freq,file_path)

ori_energy
-5.687536347860019
ori_pressure
0.6136486774822469
-------------------
0 -5.687536347860019 0.6136486774822469
-------------------
